In [1]:
%pip install selenium

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [3]:
driver = webdriver.Chrome()
driver.get("https://www.instagram.com/explore/tags/%E5%8F%B0%E5%8C%97%E7%BE%8E%E9%A3%9F/")
time.sleep(10)
for j in range(4):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.3)
time.sleep(5)
driver.find_element_by_class_name("KPnG0").click()
driver.find_element_by_id("email").send_keys("676767@tp.edu.tw")
driver.find_element_by_id("pass").send_keys("kaokao0584")
driver.find_element_by_id("loginbutton").click()
time.sleep(10)

In [4]:
def count(dict,x,tempdata):
    for contest in [tempdata['contest'],tempdata['tag']]:
        for sentence in contest:
            for i in range(len(sentence)-x-1):
                isbreak = 0
                for j in range(x):
                    for k in [' ','#', '(' , ')' , '/', '\'','\"' , '_','-','+','"' , '{' , '}' , '.' , ',' , '!' , '?' , '[' , ']' ,'。' , '，' , ' )', '（', ':', '：', '！','？' , '）','、']:
                        if sentence[i+j] == k or (sentence[i+j] <= 'z' and sentence[i+j] >= 'a') or (sentence[i+j] <= 'Z' and sentence[i+j] >= 'A') or (sentence[i+j] <= '9' and sentence[i+j] >= '0'):
                            isbreak = 1
                            break
                if isbreak == 0:
                    try:
                        dict[sentence[i:i+x]] = dict[sentence[i:i+x]] + 1
                    except:
                        dict[sentence[i:i+x]] = 1

In [5]:
def count2(dict,x,tempdata):
    for contest in [tempdata['tag']]:
        for sentence in contest:
            for i in range(len(sentence)-x-1):
                isbreak = 0
                for j in range(x):
                    for k in [' ','#', '(' , ')' , '/', '\'','\"' , '_','-','+','"' , '{' , '}' , '.' , ',' , '!' , '?' , '[' , ']' ,'。' , '，' , ' )', '（', ':', '：', '！','？' , '）','、']:
                        if sentence[i+j] == k or (sentence[i+j] <= 'z' and sentence[i+j] >= 'a') or (sentence[i+j] <= 'Z' and sentence[i+j] >= 'A') or (sentence[i+j] <= '9' and sentence[i+j] >= '0'):
                            isbreak = 1
                            break
                if isbreak == 0:
                    try:
                        dict[sentence[i:i+x]] = dict[sentence[i:i+x]] + 1
                    except:
                        dict[sentence[i:i+x]] = 1

In [6]:
def storyProcess(n,data):
    tempList = {}
    html = BeautifulSoup(driver.page_source)
    story = html.find('div',class_ = ['C4VMK'])
    if(story == None):
        tempList['tag'] = []
        tempList["contest"] = []
    else:
        # hashtag
        hashtag = story.find_all('a',class_ = 'xil3i')
        tempTag = []
        for eachTag in hashtag:
            tempTag.append(eachTag.string)
        tempList['tag'] = tempTag
        # 正文
        contest = story.find('h1')
        try: [temp.extract() for temp in contest(['a','span'])]
        except: print('')
        contestList = list(contest)
        finalContest = []
        for x in contestList:
            try:
                _ = x.type
            except:
                if(len(x) > 2):
                    finalContest.append(x)
        tempList["contest"] = finalContest
    # 加入data
    data.loc[n] = tempList

In [7]:
def oneWeb(name,data):
    print('enter ' + name)
    #-----------更改疊代次數----------------------------------------------------------------------------------------------------
    infoScale = 100000000
    time.sleep(10)
    driver.find_element_by_class_name("_9AhH0").click()
    storyProcess(0,data)
    second = driver.find_elements_by_css_selector("a[class][tabindex]")[0]
    ActionChains(driver).click(second).perform()
    for i in range(infoScale):
        time.sleep(2.5)
        if(i % 100 == 0): print(i)
        try: storyProcess(i+1,data)
        except: print('')
        try:
            second = driver.find_elements_by_css_selector("a[class][tabindex]")[1]
            ActionChains(driver).click(second).perform()
        except:
            infoScale = i+1
            print('final scale = ' + infoScale)
            break
    dropList = []
    for i in range(infoScale - 1):
        if(data.iloc[i,:]['contest'] == data.iloc[i+1,:]['contest']):
            if(data.iloc[i,:]['tag'] == data.iloc[i+1,:]['tag']):
                dropList.append(i)
    Data = data.drop(dropList)
    path =  'C:\code\python_practice\worm\_' + name + '.csv'
    Data.to_csv(path)
    Data =  pd.read_csv(path)
    dic = {}
    count(dic,2,Data)
    count(dic,3,Data)
    count(dic,4,Data)
    count(dic,5,Data)
    dic = sorted(dic.items(), key = lambda item:item[1],reverse = True)
    dicData = pd.DataFrame(dic[:500])
    path =  'C:\code\python_practice\worm\_' + name + '_dic.csv'
    dicData.to_csv(path)
    dic2 = {}
    count2(dic2,2,Data)
    count2(dic2,3,Data)
    count2(dic2,4,Data)
    count2(dic2,5,Data)
    dic2 = sorted(dic2.items(), key = lambda item:item[1],reverse = True)
    dicData2 = pd.DataFrame(dic2[:100])
    path =  'C:\code\python_practice\worm\_' + name + '_tag_dic.csv'
    dicData2.to_csv(path)

In [10]:
time.sleep(5)  
data = pd.DataFrame(columns = ['contest','tag'])
#----------更改名單----------------------------------------------------------------------------
List = ['#花蓮美食'] 
for i,name in enumerate(List):
    driver.find_element_by_class_name("XTCLo").send_keys(name)
    time.sleep(5)
    driver.find_element_by_class_name("z556c").click()
    oneWeb(name,data)
    driver.find_element_by_css_selector("path[d = 'M41.8 9.8L27.5 24l14.2 14.2c.6.6.6 1.5 0 2.1l-1.4 1.4c-.6.6-1.5.6-2.1 0L24 27.5 9.8 41.8c-.6.6-1.5.6-2.1 0l-1.4-1.4c-.6-.6-.6-1.5 0-2.1L20.5 24 6.2 9.8c-.6-.6-.6-1.5 0-2.1l1.4-1.4c.6-.6 1.5-.6 2.1 0L24 20.5 38.3 6.2c.6-.6 1.5-.6 2.1 0l1.4 1.4c.6.6.6 1.6 0 2.2z']").click()
    time.sleep(50)

enter #花蓮美食
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400


TypeError: can only concatenate str (not "int") to str

In [41]:
print('start')
name = '花蓮美食'
infoScale = 6407
dropList = []
for i in range(infoScale - 1):
    if(data.iloc[i,:]['contest'] == data.iloc[i+1,:]['contest']):
        print(data.iloc[i,:]['contest'])
        if(data.iloc[i,:]['tag'] == data.iloc[i+1,:]['tag']):
            dropList.append(i)
    elif '營業時間' in data['contest'].iloc[i] or  '地址' in data['contest'].iloc[i]:
        dropList.append(i)
Data = data.drop(dropList)
path =  'C:\code\python_practice\worm\_' + name + '.csv'
Data.to_csv(path)
Data =  pd.read_csv(path)
dic = {}
count(dic,2,Data)
count(dic,3,Data)
count(dic,4,Data)
count(dic,5,Data)
dic = sorted(dic.items(), key = lambda item:item[1],reverse = True)
dicData = pd.DataFrame(dic[:500])
path =  'C:\code\python_practice\worm\_' + name + '_dic.csv'
dicData.to_csv(path)
dic2 = {}
count2(dic2,2,Data)
count2(dic2,3,Data)
count2(dic2,4,Data)
count2(dic2,5,Data)
dic2 = sorted(dic2.items(), key = lambda item:item[1],reverse = True)
dicData2 = pd.DataFrame(dic2[:100])
path =  'C:\code\python_practice\worm\_' + name + '_tag_dic.csv'
dicData2.to_csv(path)

start
['✔️想賺錢嗎❓', '✔️我們只收一次性會費提供十二種工作🥳', '✔️每一天都有別人加入我們的團隊', '✔️我們絕對不會騙人', '✔️不再被會費低而騙走😟', '✔️曾比人騙過的 現在都賺回來了', '✔️不需要跟別人搶單做打字🤩', '✔️你還在考慮嗎？\U0001f971', '💰努力工作 我帶你賺錢', '🔍歡迎DM查詢❤️ 🌟招圖文識別🔎', '🌟招投票點贊員👍🏻', '🌟招網絡遊戲代練🎮', '🌟招快遞填單員✍🏻', '🌟招派單員📬', '🌟招代理📲', '🌟招培訓老師📜', '🌟招任務主管🎖', '🌟招原創寫作💭', '🌟招稿件錄入員✏', '🌟招微商🛍', '🌟招網絡主播🎙']
['✔️想賺錢嗎❓', '✔️我們只收一次性會費提供十二種工作🥳', '✔️每一天都有別人加入我們的團隊', '✔️我們絕對不會騙人', '✔️不再被會費低而騙走😟', '✔️曾比人騙過的 現在都賺回來了', '✔️不需要跟別人搶單做打字🤩', '✔️你還在考慮嗎？\U0001f971', '💰努力工作 我帶你賺錢', '🔍歡迎DM查詢❤️ 🌟招圖文識別🔎', '🌟招投票點贊員👍🏻', '🌟招網絡遊戲代練🎮', '🌟招快遞填單員✍🏻', '🌟招派單員📬', '🌟招代理📲', '🌟招培訓老師📜', '🌟招任務主管🎖', '🌟招原創寫作💭', '🌟招稿件錄入員✏', '🌟招微商🛍', '🌟招網絡主播🎙']
['✔️想賺錢嗎❓', '✔️我們只收一次性會費提供十二種工作🥳', '✔️每一天都有別人加入我們的團隊', '✔️我們絕對不會騙人', '✔️不再被會費低而騙走😟', '✔️曾比人騙過的 現在都賺回來了', '✔️不需要跟別人搶單做打字🤩', '✔️你還在考慮嗎？\U0001f971', '💰努力工作 我帶你賺錢', '🔍歡迎DM查詢❤️ 🌟招圖文識別🔎', '🌟招投票點贊員👍🏻', '🌟招網絡遊戲代練🎮', '🌟招快遞填單員✍🏻', '🌟招派單員📬', '🌟招代理📲', '🌟招培訓老師📜', '🌟招任務主管🎖', '🌟招原創寫作💭', '🌟招稿件錄入員✏', '🌟招微商🛍', '🌟招網絡主播🎙']
['✔️想賺錢嗎❓', '✔️我們只收一次性會費提供十二種工作🥳', '✔️每一天都有別人加入我們的團隊', '✔️我們絕對不會騙人', '✔️不再被會費低而

KeyboardInterrupt: 

In [23]:
print(data)

                                                contest  \
0                                                    []   
1     [再一發花蓮景點了📍海崖谷, 大家應該已經不陌生了, 不過😘我還是想來介紹一下, 花蓮最近真...   
2                    [差點沒被小黑蚊包圍叮死, 要來這的人記得帶大量防蚊液與長袖長褲😬]   
3        [✨富源手工水餃🥟, ▫️水餃一粒 $5, 餃系列認真愛韭菜, 雖然吃完嘴巴會臭, W/ ]   
4     [🐼花蓮市 味本物🐼, 🔺白玉湯圓紅豆湯 $60, 記得上次去沒吃到白玉湯圓, 這次我終於記...   
5     [謝謝今天來找可頌🥐的朋友們，希望在這特別的日子能讓你們有滿滿的幸福❤️, 祝大家情人節快樂...   
6     [✨人生如茶 — , ▫️百香綠茶 45, ▫️鮮粹繽紛水果茶 65, ▫️鮮榨葡萄柚蜜綠 ...   
7                             [珍煮丹新品-芝麻控不能錯過吧✨, 全台門市販售]   
8     [各位客倌，請直接爆收藏一波, 花蓮超有名的平價壽司店，最近開來羅東了！終於有機會去吃的牙籤...   
9     [即日起單筆消費🈵️500現折100！, 外帶自取皆可使用！, 台北大安店, 📍地址:台北市...   
10    [我已經開始懷念這個地平線，不知何年何月才能再來個小旅行😢, 不過今年的小心願已經完成✅, ...   
11    [✔️想賺錢嗎❓, ✔️我們只收一次性會費提供十二種工作🥳, ✔️每一天都有別人加入我們的團...   
12    [✔️想賺錢嗎❓, ✔️我們只收一次性會費提供十二種工作🥳, ✔️每一天都有別人加入我們的團...   
13    [✔️想賺錢嗎❓, ✔️我們只收一次性會費提供十二種工作🥳, ✔️每一天都有別人加入我們的團...   
14    [2020.03.29, [花蓮。卡克特司 cactus ], •鮮奶油珍珠鮮奶茶$130,...   
15                                   [一起穿過那座山, 一起越過那海洋] 

In [19]:
'營業時間' in data.iloc[8,:]['contest']

False